In [1]:
import os
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from moviepy.editor import VideoFileClip
from collections import deque
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

In [2]:
'''Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.'''
object_points=[]
image_points=[]
objp = np.zeros((6*9,3), np.float32) # 6,9 represents the total number of co-ordinates in y and x direction respectively.
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
# Iterate through the lists and get the corners of the images
#plt.figure(figsize = (16,32)) # To plot the (width, height) in inches
for image in os.listdir("camera_cal/"): # image will iterate through the list of the paths and consider each path one after another 
    img = cv2.imread("camera_cal/" + image) # img stores the data of the image located in the path in the form of numpy array and reads the data of pixel in (BGR) format.    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # gray stres the graysacle img
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None) # Prints the 9*6=54 corners (x,y) co-ordinates.
    # if all the corners are present in the gray(image) the image returns True
    if ret == True :
        object_points.append(objp) #objp are added to the object_points
        image_points.append(corners) # Corners are added to the image_points 
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(object_points, image_points, gray.shape[::-1], None, None)


In [3]:
def threshold_binary_image(img):    
    #print("img")
#     plt.imshow(img)
#     plt.show()    

    # Convert to HLS color space and separate the S channel
    # Note: img is the undistorted image
    # Grayscale image
    # NOTE: we already saw that standard grayscaling lost color information for the lane lines
    # Explore gradients in other colors spaces / color channels to see what might work better
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Sobel x and Sobel y
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = 15) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobelx = np.uint8(255*abs_sobelx/np.max(abs_sobelx)) #Rescale to unit 8
    sobely = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = 15) # Take the derivative in y
    abs_sobely = np.absolute(sobely) # Absolute y derivative to accentuate lines away from horizontal
    scaled_sobely = np.uint8(255*abs_sobely/np.max(abs_sobely)) #Rescale to unit 8

    # Threshold x gradient and y gradient
    thresh_min = 50
    thresh_max = 150
    sxbinary = np.zeros_like(scaled_sobelx) # Has similar dimensions to the scaled_sobelx but replcaes elements with 0.
    sxbinary[(scaled_sobelx >= 35) & (scaled_sobelx <= thresh_max)] = 1 # Making the points which fall in threshold to 1.
    sybinary = np.zeros_like(scaled_sobely)
    sybinary[(scaled_sobely >= thresh_min) & (scaled_sobely <= thresh_max)] = 1
    
    # Magnitude of gradient
    magnitude = np.sqrt(sobelx**2 + sobelx**2)
    scaled_sobelm = np.uint8(255*magnitude/np.max(magnitude)) 
    smbinary = np.zeros_like(magnitude) 
    smbinary[(scaled_sobelm >= thresh_min) & (scaled_sobelm <= thresh_max)] = 1 # Create a binary image of ones where threshold is met, zeros otherwise
    
    # Direction of the gradient
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0) # By increasing the size of kenrel we were able to pick up particular points
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1) # So I have taken the kernel sixe = 31 which is maximum in this case.
    direction = np.arctan2(np.absolute(sobel_y), np.absolute(sobel_x))
    sdbinary =  np.zeros_like(direction)
    sdbinary[(direction >= 0.7) & (direction <= 1.0)] = 1
    
    # Compute the combination of Sobel X and Sobel Y or Magnitude and Direction
    sxymd_binary = np.zeros_like(sdbinary)
    sxymd_binary[(sxbinary == 1) | ((sybinary == 1) & (smbinary == 1) & (sdbinary == 1))] = 1
    
    
#     # Saving the threshold binary image  
#     cv2.imwrite("output_images/Threshold_binary_images/"+i, binary*255)
    
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV) # Converting image from BGR to HLS channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    yellow = np.zeros_like(hls[:,:,0])
    yellow[((hls[:,:,0] >= 15) & (hls[:,:,0] <= 35)) & ((hls[:,:,1] >= 30) & (hls[:,:,1] <= 204)) & ((hls[:,:,2] >= 115) & (hls[:,:,2] <= 255))] = 1
    
    white = np.zeros_like(hls[:,:,0])
    white[((hls[:,:,0] >= 0) & (hls[:,:,0] <= 255)) & ((hls[:,:,1] >= 200) & (hls[:,:,1] <= 255)) & ((hls[:,:,2] >= 0) & (hls[:,:,2] <= 255))] = 1               
    
#     channel = np.zeros_like(white) 
#     channel[(white == 1) | (yellow == 1)] = 1
    

    #Combine the two binary thresholds
    combined_binary = np.zeros_like(sxymd_binary)
    combined_binary[(white == 1) & (sxymd_binary == 1)] = 1
    
    binary = np.zeros_like(sxymd_binary)
    binary[(combined_binary == 1) | (yellow == 1)] = 1
    
#     print("binary")
#     plt.imshow(binary)
#     plt.show()    
    return binary
    

In [4]:
def perspective_transform(img):
    

    
    width = img.shape[1] # Width of the image
    height = img.shape[0] # Height of the image
    # Points selected on trial and error basis
    src = np.array([[210,height],[595,450],[690,450], [1110, height]], dtype=np.float32)
    offset = 200
    dst = np.array([[offset, height],[offset, 0],[width-offset, 0], [width-offset, height]], dtype=np.float32)
    # Perspective transform
    M = cv2.getPerspectiveTransform(src, dst)
    # Inverse perspective transform
    Minv = cv2.getPerspectiveTransform(dst, src)
    # Warped output
    warped = cv2.warpPerspective(img, M, (width, height), flags=cv2.INTER_LINEAR)
#     print("warped")
#     plt.imshow(warped)
#     plt.show()
    return Minv, warped
    
    

In [15]:
def detect_lanes(binary_warped):
#     plt.imshow(binary_warped)
#     plt.show()

#     plt.imshow(binary_warped)
#     plt.show()
    # Take a histogram of the bottom half of the image
    bottom_half_y = binary_warped.shape[0]/2
    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    #plt.plot(histogram)
#     plt.plot(histogram)
#     plt.show()
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    #print(midpoint,leftx_base,rightx_base)
    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
        (0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),
        (0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        #print(good_left_inds,good_right_inds)
        # Append these indices to the lists
        #print(good_left_inds)
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
    # Concatenate the arrays of indices
    #print(left_lane_inds)
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds) 
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]    
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    return left_fit,right_fit

In [ ]:
def previous_detect_lanes(warped, left_fit, right_fit):
    # Assume you now have a new warped binary image 
    # from the next frame of video (also called "binary_warped")
    # It's now much easier to find line pixels!
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)  
    return left_fit,right_fit,left_lane_inds,right_lane_inds

In [6]:
def curvature_lane(left_fit, right_fit, image):

    ploty = np.linspace(0, image.shape[0]-1, image.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    y_eval = np.max(ploty)
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)    
    # Calculate the new radii of curvature
    left_curve = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curve = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Pixels to meters
    x_max = img.shape[1]*xm_per_pix
    y_max = img.shape[0]*ym_per_pix
    vehicle_center = x_max/2
    # Calculate the vehicle position with respect to center
    center_left = left_fit[0]*y_max**2 + left_fit[1]*y_max + left_fit[2]
    center_right = right_fit[0]*y_max**2 + right_fit[1]*y_max + right_fit[2]
    center = center_left + (center_right - center_left)/2
    center_difference = center - vehicle_center
    return left_curve, right_curve, center_difference
   

In [7]:
def warp_original(left_fit, right_fit, Minv, binary_warped, undistorted):
    
    # Create an image to draw the lines on
    #binary_warped = cv2.cvtColor(binary_warped, cv2.COLOR_BGR2GRAY)
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (binary_warped.shape[1], binary_warped.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undistorted, 1, newwarp, 0.3, 0)
    return result

In [8]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False        
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None

In [12]:
left_line = Line()
right_line = Line()

In [9]:
def pipeline(image):
    '''Apply a distortion correction to raw image.'''
    undistorted = cv2.undistort(image, mtx, dist, None, mtx)
    '''Use color transforms, gradients, etc., to create a thresholded binary image.'''
    combined_binary = threshold_binary_image(undistorted)
    '''Apply a perspective transform to rectify binary image ("birds-eye view").'''
    Minv, warped = perspective_transform(combined_binary)
    '''Detect lane pixels and fit to find the lane boundary.'''
    if left_line.detected  == False or right_line.detected == False:
        left_fit, right_fit, left_lane_inds, right_lane_inds = detect_lanes(warped)
        
    else:
        left_fit, right_fit, left_lane_inds, right_lane_inds = previous_detect_lanes(warped, left_fit.best_fit, right_fit.best_fit)
    if left_fit is not None and right_fit is not None:
        # calculate x-intercept (bottom of image, x=image_height) for fits
        y = img.shape[0]
        left_fitx = left_fit[0]*y**2 + left_fit[1]*y + left_fit[2]
        right_fitx = [0]*y**2 + right_fit[1]*y + right_fit[2]
        initial_points_difference = abs(right_fitx-left_fitx)
        if abs(350 - initial_points_difference) > 100:
            left_fit = None
            right_fit = None
            
    left_line.add_fit(left_fit, l_lane_inds)
    right_line.add_fit(right_fit, r_lane_inds)    
    
    
    '''Determine the curvature of the lane and vehicle position with respect to center.'''
    left_curve, right_curve, center_difference = curvature_lane(left_fit, right_fit, warped)
    '''Warp the detected lane boundaries back onto the original image.'''
    final_image = warp_original(left_fit, right_fit, Minv, warped, undistorted)
    cv2.putText(final_image,str((left_curve+right_curve)/2) , (100, 90), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), thickness=2)
    # Returning the final image
    return final_image

# '''Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position'''
# for i in os.listdir("test_images/"):
#     image = plt.imread("test_images/" + i)
#     processed_image= pipeline(image)
#     plt.imshow(processed_image)
#     plt.show()
#     cv2.imwrite("output_images/test_images/"+i, processed_image)    
    
    
    
    
    
    

In [ ]:
white_output = 'Output_videos/project_video.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(pipeline) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [16]:
white_output = 'Output_videos/challenge_video.mp4'
clip1 = VideoFileClip("challenge_video.mp4")
white_clip = clip1.fl_image(pipeline) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

UnboundLocalError: local variable 'left_fit' referenced before assignment

In [ ]:
white_output = 'Output_videos/harder_challenge_video.mp4'
clip1 = VideoFileClip("harder_challenge_video.mp4")
white_clip = clip1.fl_image(pipeline) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [ ]:
# best_fit_test_images = glob.glob('output_images/test*_top_view.jpg')
# n_frames = 0

# def video_pipeline(img):
#     global n_frames
#     if n_frames == 0:
#         undistorted = undistort(img)
#         binary_thresholded = combined_thresholds(undistorted)
#         warped, M, Minv = transform_perspective(binary_thresholded)
#         windows_img, ploty, left_fitx, right_fitx, left_fit, right_fit, leftx, rightx, leftx_base, rightx_base = sliding_windows(warped)   
#         left_curverad, right_curverad = roc_in_meters(ploty, left_fit, right_fit, leftx, rightx)
#         camera = offset(img.shape[1]/2, leftx_base, rightx_base)
#         out_img = lane_mapping(img.shape[:2], warped, Minv, ploty, left_fitx, right_fitx, undistorted)
#         annotate(out_img, left_curverad, right_curverad, camera)
#     else:
#         undistorted = undistort(img)
#         binary_thresholded = combined_thresholds(undistorted)
#         region = region_of_interest(binary_thresholded)
#         warped, M, Minv = transform_perspective(region)
#         binary_warped = warped
        
#         if LEFT.detected == True:
#             left_fit = LEFT.current_fit_coeffs
#         else:
#             left_fit = LEFT.best_fit_coeffs
        
#         if RIGHT.detected == True:
#             right_fit = RIGHT.current_fit_coeffs
#         else:
#             right_fit = RIGHT.best_fit_coeffs
        
#         windows_img, ploty, left_fitx, right_fitx, left_fit, right_fit, leftx, rightx, leftx_base, rightx_base = sliding_windows(warped)   

#         if LEFT.detected == True:
#             LEFT.recent_xfits.append((left_fitx))
#             LEFT.best_xfit = np.mean(LEFT.recent_xfits)
#             LEFT.current_fit_coeffs.append((left_fit))
#             LEFT.best_fit_coeffs = np.mean(LEFT.current_fit_coeffs)
#             LEFT.line_base_position = leftx_current
#             LEFT.allx = leftx
#             LEFT.ally = lefty
#         else:
#             left_fitx = LEFT.best_xfit
#             left_fit = LEFT.best_fit_coeffs
#             leftx_current = LEFT.line_base_position
#             leftx = LEFT.allx
#             lefty = LEFT.ally   
#         if RIGHT.detected == True:
#             RIGHT.recent_xfits.append((right_fitx))
#             RIGHT.best_xfit = np.mean(RIGHT.recent_xfits)
#             RIGHT.current_fit_coeffs.append((right_fit))
#             RIGHT.best_fit_coeffs = np.mean(RIGHT.current_fit_coeffs)
#             RIGHT.line_base_position = rightx_current
#             RIGHT.allx = rightx
#             RIGHT.ally = righty   
#         else:
#             right_fitx = RIGHT.best_xfit
#             right_fit = RIGHT.best_fit_coeffs
#             rightx_current = RIGHT.line_base_position
#             rightx = RIGHT.allx
#             righty = RIGHT.ally
            
#         is_lane_width_correct = verify_lane_width(LEFT.line_base_position, RIGHT.line_base_position)
#         if is_lane_width_correct == True:
#             LEFT.detected = True
#             RIGHT.detected = True
#         else:
#             LEFT.detected = False
#             RIGHT.detected = False
                
#         left_curverad, right_curverad =  roc_in_meters(ploty, left_fit, right_fit, leftx, rightx)
#         LEFT.radius_of_curvature = left_curverad
#         RIGHT.radius_of_curvature = right_curverad
        
#         leftx_base = leftx_current
#         rightx_base = rightx_current
#         camera = offset(img.shape[1]/2, leftx_base, rightx_base, n_frames)
        
#         out_img = lane_mapping(img.shape[:2], warped, Minv, ploty, left_fitx, right_fitx, undistorted)
#         annotate(out_img, left_curverad, right_curverad, camera)    
#     return out_img